In [4]:
from torch.utils.data import Dataset, DataLoader
import torch
import torch.nn as nn

In [5]:
x = [[1,2],[3,4],[5,6],[7,8]]
y = [[3],[7],[11],[15]]

In [6]:
X = torch.tensor(x).float()
Y = torch.tensor(y).float()

In [8]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
X = X.to(device)
Y = Y.to(device)

In [9]:
class MyDataset(Dataset):
  def __init__(self,x,y):
    self.x = torch.tensor(x).float()
    self.y = torch.tensor(y).float()
  def __len__(self):
    return len(self.x)
  def __getitem__(self, ix):
    return self.x[ix], self.y[ix]

In [10]:
ds = MyDataset(X, Y)

/tmp/ipython-input-2407655035.py:3: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.x = torch.tensor(x).float()
/tmp/ipython-input-2407655035.py:4: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.y = torch.tensor(y).float()


In [11]:
dl = DataLoader(ds, batch_size=2, shuffle=True)

In [12]:
from torch import nn
class MyNeuralNet(nn.Module):
  def __init__(self):
    super().__init__()
    self.input_to_hidden_layer = nn.Linear(2,8)
    self.hidden_layer_activation = nn.ReLU()
    self.hidden_to_output_layer = nn.Linear(8,1)
  def forward(self, x):
    x = self.input_to_hidden_layer(x)
    x = self.hidden_layer_activation(x)
    x = self.hidden_to_output_layer(x)
    return x

In [13]:
mynet = MyNeuralNet().to(device)

In [14]:
# define loss function
loss_fn = nn.MSELoss()

In [15]:
optimizer = torch.optim.SGD(mynet.parameters(), lr=0.001)

In [18]:
# perform weight update for 50 epochs
loss_values = []
for _ in range(50):
  for data in dl:
    X, Y = data
    # flush the previous epoch's gradients
    optimizer.zero_grad()
    # make a prediction
    pred = mynet(X)
    # calculate the loss
    loss = loss_fn(pred, Y)
    #perform backpropagation
    loss.backward()
    # updae the weightd according to the gradients computed
    optimizer.step()
    # append loss values
    loss_values.append(loss.item())

#### predicting on new datapoint

In [20]:
val_x = [[10, 11]]
val_x = torch.tensor(val_x).float()

In [21]:
mynet(val_x)

tensor([[21.2519]], grad_fn=<AddmmBackward0>)